In [45]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import ast, os

In [46]:
if not os.path.exists("data_cleaned.csv"):
    data = pd.read_csv("CarGroup/0.csv",header=None)
    column_names = [
        'vin','数据采集时间', '车辆状态', '充电状态', '运行模式', '车速', '累积里程', '总电压', '总电流',
        'SOC', 'DC-DC状态', '档位', '绝缘电阻', '经度', '纬度', '电池单体电压最高值',
        '电池单体电压最低值', '最高温度值', '最低温度值', '驱动电机序号', '驱动电机转速', '驱动电机转矩'
    ]

    data.columns = column_names

    # 主数据预处理
    # 处理数值数据的有效范围和异常值，删除包含不符合要求的值的行
    data['车速'] = data['车速'].apply(lambda x: x * 0.1 if 0 <= x <= 2200 else None)  # 有效范围0~2200
    data['累积里程'] = data['累积里程'].apply(lambda x: x * 0.1 if 0 <= x <= 9999999 else None)  # 有效范围0~9999999
    data['总电压'] = data['总电压'].apply(lambda x: x * 0.1 if 0 <= x <= 10000 else None)  # 有效范围0~10000
    data['总电流'] = data['总电流'].apply(lambda x: (x - 1000) * 0.1 if 0 <= x <= 20000 else None)  # 有效范围-1000A~1000A
    data['SOC'] = data['SOC'].apply(lambda x: x if 0 <= x <= 100 else None)  # 有效范围0~100
    data['绝缘电阻'] = data['绝缘电阻'].apply(lambda x: x * 1000 if 0 <= x <= 60000 else None)  # 有效范围0~60000
    data['电池单体电压最高值'] = data['电池单体电压最高值'].apply(lambda x: x * 0.001 if 0 <= x <= 15000 else None)  # 有效范围0~15000
    data['电池单体电压最低值'] = data['电池单体电压最低值'].apply(lambda x: x * 0.001 if 0 <= x <= 15000 else None)  # 有效范围0~15000
    data['最高温度值'] = data['最高温度值'].apply(lambda x: x - 40 if 0 <= x <= 250 else None)  # 有效范围-40°C~+210°C
    data['最低温度值'] = data['最低温度值'].apply(lambda x: x - 40 if 0 <= x <= 250 else None)  # 有效范围-40°C~+210°C
    data['驱动电机转速'] = data['驱动电机转速'].apply(lambda x: x - 20000 if 0 <= x <= 65531 else None)  # 有效范围-20000r/min~45531r/min
    data['驱动电机转矩'] = data['驱动电机转矩'].apply(lambda x: (x - 2000) * 0.1 if 0 <= x <= 65531 else None)  # 有效范围-2000N·m~4553.1N·m
    
    # 将时间转为时间戳
    data['数据采集时间'] = pd.to_datetime(data['数据采集时间'])

    # 删除包含任何None值的行
    data.dropna(axis=0, how='any', inplace=True)
    data.to_csv("data_cleaned.csv",index=False)
else: 
    data = pd.read_csv("data_cleaned.csv")

In [47]:
data_trip = pd.read_csv('trip_data.csv')
data_trip = data_trip[(data_trip['trip_kind'] != 'P') & (data_trip['trip_kind'] != 'C')]
# data_trip = data_trip[:20]
data_trip = data_trip.reset_index(drop=True)
data_trip['Unnamed: 0'] = data_trip.index
# 将时间转为时间戳
data_trip['start_collectiontime'] = pd.to_datetime(data_trip['start_collectiontime'])
data_trip['end_collectiontime'] = pd.to_datetime(data_trip['end_collectiontime'])
data_trip

,Unnamed: 0,vin,trip_kind,start_collectiontime,end_collectiontime
0,0,b2b30b19de1ee6575fcd61f277fb852f,D,2020-11-30 07:10:00,2020-11-30 09:46:00
1,1,7441ab176d801a8267fde2738a24363e,D,2020-10-22 08:52:00,2020-10-22 08:59:00
2,2,e145dfeff3096c502d648a726770cba2,D,2020-10-20 23:34:00,2020-10-21 00:24:00
3,3,e145dfeff3096c502d648a726770cba2,D,2020-11-12 14:26:00,2020-11-12 14:52:00
4,4,e145dfeff3096c502d648a726770cba2,D,2020-11-22 12:10:00,2020-11-22 12:25:00
...,...,...,...,...,...
483780,483780,eda9873a9cc3ff503fafa7458e4ba567,D,2020-11-07 02:06:00,2020-11-07 02:09:00
483781,483781,9cea1f63ba31dc15ed61f24373b9bff5,D,2020-10-18 16:09:00,2020-10-18 17:44:00
483782,483782,df18eb382dc0d23ce75c261d1a534652,D,2020-10-07 13:30:00,2020-10-07 14:27:00
483783,483783,9cea1f63ba31dc15ed61f24373b9bff5,D,2020-12-06 09:12:00,2020-12-06 10:45:00


In [48]:
mini_data = data[data['vin'].isin(data_trip['vin'])]
mini_data = mini_data.reset_index(drop=True)
mini_data

,vin,数据采集时间,车辆状态,充电状态,运行模式,车速,累积里程,总电压,总电流,SOC,...,绝缘电阻,经度,纬度,电池单体电压最高值,电池单体电压最低值,最高温度值,最低温度值,驱动电机序号,驱动电机转速,驱动电机转矩
0,000cdb8748c791b5b57f2fe4c5030f71,2020-11-01 08:09:49,2,3,1,0.0,64897.0,560.0,0.2,100,...,5858,121.33633,30.77259,3.669,3.436,35,32,[1],[-15000],[-500.0]
1,000cdb8748c791b5b57f2fe4c5030f71,2020-11-01 08:10:19,1,3,1,45.0,64899.0,527.0,-16.0,95,...,7268,121.34016,30.77230,3.310,3.289,30,27,[1],[2265],[-54.5]
2,000cdb8748c791b5b57f2fe4c5030f71,2020-11-01 08:10:49,1,3,1,21.0,64899.0,524.0,-3.1,94,...,7511,121.34265,30.77181,3.294,3.267,30,27,[1],[1078],[-48.0]
3,000cdb8748c791b5b57f2fe4c5030f71,2020-11-01 08:11:19,1,3,1,40.0,64899.0,508.0,50.1,93,...,6823,121.34280,30.76931,3.213,3.149,30,27,[1],[2457],[75.6]
4,000cdb8748c791b5b57f2fe4c5030f71,2020-11-01 08:11:49,1,3,1,52.0,64900.0,514.0,48.1,93,...,8649,121.34597,30.76920,3.241,3.182,30,27,[1],[3068],[67.4]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8708986,0ff938307317815a9feb862ffadde60b,2020-12-31 23:29:41,2,1,2,0.0,39512.0,370.3,-0.9,100,...,2603,0.00000,0.00000,1.000,1.000,-40,-40,NaN,NaN,NaN
8708987,0ff938307317815a9feb862ffadde60b,2020-12-31 23:29:51,2,1,2,0.0,39512.0,370.3,-1.0,100,...,2603,0.00000,0.00000,1.000,1.000,-40,-40,NaN,NaN,NaN
8708988,0ff938307317815a9feb862ffadde60b,2020-12-31 23:30:01,2,1,2,0.0,39512.0,370.3,-1.0,100,...,2603,0.00000,0.00000,1.000,1.000,-40,-40,NaN,NaN,NaN
8708989,0ff938307317815a9feb862ffadde60b,2020-12-31 23:30:11,2,1,2,0.0,39512.0,370.3,-1.0,100,...,2603,0.00000,0.00000,1.000,1.000,-40,-40,NaN,NaN,NaN


In [49]:
# 对于 mini_data中的每一行，在data_trip中寻找是否有vin相同，且数据采集时间位于start_collectiontime，end_collectiontime的行。如果没有，则删除该行；如果有，在该行后添加一列，值为data_trip对应行的index

In [50]:
for i, row in mini_data.iterrows():
    vin = row['vin']
    collectiontime = row['数据采集时间']
    trip = data_trip[(data_trip['vin'] == vin) & (data_trip['start_collectiontime'] <= collectiontime) & (data_trip['end_collectiontime'] >= collectiontime)]
    if len(trip) == 0:
        mini_data.drop(i, inplace=True)
    else:
        trip_index = trip.index[0]
        mini_data.loc[i, 'trip_index'] = trip_index

KeyboardInterrupt: 

In [ ]:
mini_data

,vin,数据采集时间,车辆状态,充电状态,运行模式,车速,累积里程,总电压,总电流,SOC,...,绝缘电阻,经度,纬度,电池单体电压最高值,电池单体电压最低值,最高温度值,最低温度值,驱动电机序号,驱动电机转速,驱动电机转矩


In [ ]:
# # merge data and data_type dataframes on 'vin' column
# merged_data = pd.merge(data, data_trip, on='vin')

# # filter rows based on collectiontime being between start_collectiontime and end_collectiontime
# filtered_data = merged_data[(merged_data['collectiontime'] >= merged_data['start_collectiontime']) & (merged_data['collectiontime'] <= merged_data['end_collectiontime'])]

# # drop the start_collectiontime and end_collectiontime columns
# filtered_data = filtered_data.drop(['start_collectiontime', 'end_collectiontime'], axis=1)

# # display the filtered data
# filtered_data


In [ ]:
num_unique_vin = mini_data['vin'].nunique()
print(f"There are {num_unique_vin} unique 'vin' in mini_data.")


There are 0 unique 'vin' in mini_data.
